Set up the Intelligent Plant clients.

In [ ]:
import intelligent_plant.app_store_client as app_store_client
import os
app_store = app_store_client.AppStoreClient(os.environ["ACCESS_TOKEN"])
data_core = app_store.get_data_core_client()

In [ ]:
import intelligent_plant.utility as utility
import matplotlib.pyplot as plt

Import the MLP regressor from Scikit-learn

Make sure you restart the kernel after installing

In [ ]:
from sklearn.neural_network import MLPRegressor

We'll query all of the tags in IP Datasource 2 and make a list of just their names

In [ ]:
dsn = "IP Datasource 2"

In [ ]:
tags = data_core.get_tags(dsn, 1, 113)

tag_names = list(map(lambda x: x["Id"], tags))

In [ ]:
tag_names

In [ ]:
all_data = data_core.get_raw_data({dsn: tag_names}, "2014-03-03", "2014-03-03", 239)

Convert the returned data into a data frame to make it easier to work with

In [ ]:
all_data_frame = utility.query_result_to_data_frame(all_data)

In [ ]:
all_data_frame

In [ ]:
all_data_frame.plot(x="TimeStamp", legend=False)
plt.show()

To make the data easier to process we need to normalise it

In [ ]:
import sklearn.preprocessing
from pandas import DataFrame

In [ ]:
scaler = sklearn.preprocessing.MinMaxScaler()
scaled_values = scaler.fit_transform(all_data_frame.loc[:, all_data_frame.columns != 'TimeStamp'])
normalised_df = DataFrame(scaled_values)

In [ ]:
normalised_df

In [ ]:
normalised_df[:10].plot(legend=False)

In [ ]:
training_input = normalised_df.iloc[:-10,:-1]
training_output = normalised_df.iloc[:-10,-1]

testing_input = normalised_df.iloc[-10:,:-1]
testing_output = normalised_df.iloc[-10:,-1]

In [ ]:
plt.plot(training_input[:10])

The MLP is going to try and learn the relationship between the graph above and below

In [ ]:
plt.plot(training_output[:10])

In [ ]:
mlp = MLPRegressor(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5,2), random_state=1)

In [ ]:
mlp.fit(training_input, training_output)

In [ ]:
prediction = mlp.predict(testing_input)

In [ ]:
prediction_df = DataFrame({ "actual": testing_output, "prediction": prediction })

In [ ]:
prediction_df.plot()